# PipeLine
- 데이터를 크롤링하고 정제하여 DB에 저장하는 PipeLine

1. 유저에 대해 DB에 들어갈 내용을 크롤링.

2. 크롤링한 데이터를 정제.

3. 그 내용을 DB에 Insert 또는 Update
-> 기존에 존재하는 value라면, Update,
-> 기존에 존재하지 않는 value라면, Insert

# DB 테이블

1. 유저의 정보를 담은 테이블. - 유저 ID, 유저가 푼 문제 리스트, 유저 Tier, 전체 랭킹(solved.ac 사용하지 않는 사람의 티어를 예측하기 위함), 푼 문제 개수
2. 문제의 정보를 담은 테이블. - 문제 ID, 문제의 여러 정보들, 문제 context
3. 유저 - 문제 상호작용 테이블. - 유저 ID_문제 ID 키(Insert, Update를 위함), 유저 ID, 문제 ID, 맞은 횟수, 틀린 횟수, 시간 초과, 메모리 초과, 해결 언어, 사용 메모리, 걸린 시간
4. 유저 모델 예측 결과 테이블 - 유저 ID, 유저 취약 알고리즘, FFM 모델 결과 추천 리스트

위의 PipeLine에서는 1,2,3 을 갱신하는 것을 목표로 한다.


# 기존에 크롤링한 내용 DB에 집어넣기.

In [1]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re
from sklearn.preprocessing import LabelEncoder
import joblib
from collections import Counter
from collections import defaultdict
import ast
import mysql.connector

from selenium.common.exceptions import StaleElementReferenceException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [2]:
data_path = '../DataSet/'

user_df = pd.read_csv(os.path.join(data_path, 'Baekjoon_유저_크롤링.csv'))
problem_df = pd.read_csv(os.path.join(data_path, 'DB_problem_table.csv'))
user_solvedac_df = pd.read_csv(os.path.join(data_path, 'solved.ac_유저_크롤링.csv'))

In [3]:
problem_df

,problem_number,problem_name,problem_tier,problem_algorithm,problem_time_condition,problem_memory_condition,problem_submission_count,problem_answer_submission_count,problem_answered_people_count,problem_correct_rate,problem_avg_try,cat_problem_avg_try,problem_description,problem_input,problem_output
0,1010,다리 놓기,6,"['조합론', '다이나믹 프로그래밍', '수학']",0.5,128,88923,41540,33782,48.084,2.07,2,재원이는 한 도시의 시장이 되었다. 이 도시에는 도시를 동쪽과 서쪽으로 나누는 큰 ...,입력의 첫 줄에는 테스트 케이스의 개수 T가 주어진다. 그 다음 줄부터 각각의 테스...,각 테스트 케이스에 대해 주어진 조건하에 다리를 지을 수 있는 경우의 수를 출력한다.
1,1094,막대기,6,"['비트마스킹', '수학']",2.0,128,29308,21683,18316,75.244,1.33,1,"지민이는 길이가 64cm인 막대를 가지고 있다. 어느 날, 그는 길이가 Xcm인 막...",첫째 줄에 X가 주어진다. X는 64보다 작거나 같은 자연수이다.,"문제의 과정을 거친다면, 몇 개의 막대를 풀로 붙여서 Xcm를 만들 수 있는지 출력한다."
2,1181,단어 정렬,6,"['정렬', '문자열']",2.0,256,163610,68439,51305,40.323,2.48,2,알파벳 소문자로 이루어진 N개의 단어가 들어오면 아래와 같은 조건에 따라 정렬하는 ...,"첫째 줄에 단어의 개수 N이 주어진다. (1 ≤ N ≤ 20,000) 둘째 줄부터 ...",조건에 따라 정렬하여 단어들을 출력한다.
3,1193,분수찾기,6,"['구현', '수학']",0.5,256,113582,57335,49269,51.490,1.94,2,무한히 큰 배열에 다음과 같이 분수들이 적혀있다.\n1/1 1/2 1/3 1/4 1...,"첫째 줄에 X(1 ≤ X ≤ 10,000,000)가 주어진다.",첫째 줄에 분수를 출력한다.
4,1251,단어 나누기,6,"['브루트포스 알고리즘', '구현', '정렬', '문자열']",2.0,128,11073,4633,3992,46.007,2.16,2,알파벳 소문자로 이루어진 단어를 가지고 아래와 같은 과정을 해 보려고 한다.\n먼저...,첫째 줄에 영어 소문자로 된 단어가 주어진다. 길이는 3 이상 50 이하이다.,첫째 줄에 구하고자 하는 단어를 출력하면 된다.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,13705,Ax+Bsin(x)=C,21,"['임의 정밀도 / 큰 수 연산', '이분 탐색', '수학', '수치해석']",2.0,512,17330,1173,508,4.026,25.01,25,"A, B, C가 주어졌을 때, Ax+Bsin(x)=C를 만족하는 x를 찾는 프로그램...","첫째 줄에 정수 A, B, C가 주어진다. (0 < B ≤ A ≤ 100,000, ...",첫째 줄에 x를 반올림해서 소수점 여섯째 자리까지 출력한다.
6937,13926,"gcd(n, k) = 1",21,"['오일러 피 함수', '수학', '밀러–라빈 소수 판별법', '정수론', '폴라드...",2.0,512,2257,668,536,33.333,2.94,3,"자연수 n이 주어졌을 때, gcd(n, k) = 1을 만족하는 자연수 1 ≤ k ≤...",첫째 줄에 자연수 n (1 ≤ n ≤ 1018)이 주어진다.,"gcd(n, k) = 1을 만족하는 자연수 1 ≤ k ≤ n 의 개수를 출력한다."
6938,19540,인버스 ㄷㄷㄷㅈ,21,"['해 구성하기', '트리']",2.0,1024,211,101,67,47.857,2.09,2,"UCPC 출제진들은 D번:ㄷㄷㄷㅈ 문제의 데이터를 만들던 중, 정점이 많은 DUDU...",첫 번째 줄에 트리의 정점 수\nN\n$N$이 주어진다. (\n6\n≤\nN\n≤\...,N\n−\n1\n$N-1$개의 줄에 간선의 양 끝 점을 공백으로 구분해서 출력한다....
6939,26036,제곱 ㄱㄱ,21,"['임의 정밀도 / 큰 수 연산', '해 구성하기', '수학', '정수론']",1.0,512,71,33,22,64.706,1.52,2,BOJ의 쉬운 문제 제곱 ㄴㄴ를 풀지 못해 화가 난 명준은 제곱ㄴㄴ수를 뒤집은 제곱...,첫째 줄에 정수 N이 주어진다. (0 ≤ N < 1000),"A와 A+1 가 모두 제곱ㄱㄱ수이고, 100N ≤ A < 100N+1 을 만족하는 ..."


In [4]:
"""
# 문제 정보 DB에 저장.
import mysql.connector

conn = mysql.connector.connect(
    user='root',
    password='llsy13579!',
    host='localhost',
    port=3306,
    database='recommend_project'
)

DB_table_name = 'recommend_project'

cursor = conn.cursor()
use_query = f"USE {DB_table_name}"

cursor.execute(use_query)

create_problem_table_query = "CREATE TABLE IF NOT EXISTS problem_information_table ( \
    problem_number INT PRIMARY KEY, \
    problem_name VARCHAR(1000), \
    problem_tier INT, \
    problem_algorithm VARCHAR(1000), \
    problem_time_condition DOUBLE, \
    problem_memory_condition INT, \
    problem_submission_count INT, \
    problem_answer_submission_count INT, \
    problem_answered_people_count INT, \
    problem_correct_rate DOUBLE, \
    problem_avg_try DOUBLE, \
    cat_problem_avg_try INT, \
    problem_description TEXT, \
    problem_input TEXT, \
    problem_output TEXT \
);"
cursor.execute(create_problem_table_query)

for index, value in problem_df.iterrows():
    insert_query = "INSERT INTO problem_information_table (problem_number, problem_name, problem_tier, problem_algorithm, problem_time_condition, problem_memory_condition, problem_submission_count, problem_answer_submission_count, problem_answered_people_count, problem_correct_rate, problem_avg_try, cat_problem_avg_try, problem_description, problem_input, problem_output) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
    cursor.execute(insert_query, (value[0], value[1], value[2], value[3], value[4], value[5], value[6], value[7], value[8], value[9], value[10], value[11], value[12], value[13], value[14]))

conn.commit()
cursor.close()
conn.close()
"""

# DB에 문제 정보 저장 완료.

1010
1094
1181
1193
1251
1308
1312
1316
1340
1343
1380
1384
1402
1417
1418
1427
1436
1439
1475
1476
1531
1543
1652
1676
1730
1769
1789
1817
1996
2018
2057
2097
2161
2167
2238
2303
2508
2535
2545
2563
2622
2628
2635
2669
2714
2740
2751
2822
2828
2890
2891
2941
2993
3049
3054
3060
3135
3230
3699
3711
3724
3863
4108
4158
4335
4383
4446
4659
4673
4821
4884
5046
5347
5426
5462
5555
5635
5648
5671
5800
6417
6463
6504
6550
7568
7656
7785
8611
8892
8979
9037
9196
9237
9324
9358
9625
9626
9655
9733
10384
10431
10469
10709
10814
10815
10826
10867
10914
11004
11008
11059
11068
11116
11256
11292
11504
11536
11576
11637
11650
11651
11723
11728
11866
11931
11971
12033
12034
12437
12760
12787
12871
13119
13133
13241
13301
13333
13717
13909
14490
14492
14493
14582
14606
14608
14646
14654
14655
14709
14717
14721
14729
14732
14842
14912
14916
15312
15549
15687
15688
15702
15707
15719
15720
15721
15739
15779
15803
15876
15887
15904
15905
16208
16212
16218
16395
16435
16506
16961
16969
17091
17127
17176
1

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re

from selenium.common.exceptions import StaleElementReferenceException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')